# RSA with padding

Create a Python program that will be able to encrypt and decrypt the contents of any given file using PKCS #1 v1.5 or OAEP padding (both must be implemented). The encrypt and decrypt functions should be based on the cryptography library.

The public and private keys to be used can be given as input by the user or if any input regarding the public and private keys is missing, the program should be able to generate them using the function rsa.generate_private_key from the cryptography library and save them in the files public_key.pem and private_key.pem respectively.

Your program should support the following arguments:

● -e plaintext message file: Encrypt file contents

● -d encrypted message file: Decrypt file contents

● -pub public key file: File containing the public key

● -prv private key file: File containing the private key

● -p PKCS or OAEP: Padding to be used

● -o output file: Output file, if both encryption and decryption are specified, this file should contain both outputs. If no output file is specified the program should print on the console.


Verify that the ciphertext for both padding schemes causes the output to change every time, by testing your program with the same input.


Why an adversary cannot execute the chosen ciphertext attack?


Why an adversary cannot use RSA’s deterministic encryption to analyze message patterns,
frequency, and so forth

In [ ]:
import argparse
import sys
from cryptography.hazmat.primitives import serialization, hashes
from cryptography.hazmat.primitives.serialization import load_pem_private_key, load_pem_public_key
from cryptography.hazmat.primitives.asymmetric.rsa import generate_private_key, RSAPublicKey, RSAPrivateKey
from cryptography.hazmat.primitives.asymmetric import padding


In [ ]:


def encrypt_PKCS(plain: bytes, public_key: RSAPublicKey, block_size: int | None = None):
    if block_size is None:
        block_size = (public_key.public_numbers().n.bit_length()+7)//8-11

    print(block_size)

    blocks = [plain[i:i+block_size]for i in range(0, len(plain), block_size)]

    return b''.join([public_key.encrypt(block, padding.PKCS1v15()) for block in blocks])


def encrypt_OAEP(plain: bytes, public_key: RSAPublicKey, block_size: int | None = None):
    if block_size is None:
        block_size = (public_key.public_numbers(
        ).n.bit_length()+7)//8-2*256//8-2

    blocks = [plain[i:i+block_size] for i in range(0, len(plain), block_size)]

    return b''.join([public_key.encrypt(block, padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None)) for block in blocks])


def decrypt_PKCS(cipher: bytes, priv_key: RSAPrivateKey, block_size: int | None = None):
    if block_size is None:
        block_size = (priv_key.public_key(
        ).public_numbers().n.bit_length() + 7)//8

    blocks = [cipher[i:i+block_size]
              for i in range(0, len(cipher), block_size)]

    return b''.join([priv_key.decrypt(block, padding.PKCS1v15()) for block in blocks])


def decrypt_OAEP(cipher: bytes, priv_key: RSAPrivateKey, block_size: int | None = None):
    if block_size is None:
        block_size = (priv_key.public_key().public_numbers().n.bit_length()
                      + 7)//8

    blocks = [cipher[i:i+block_size]
              for i in range(0, len(cipher), block_size)]

    return b''.join([priv_key.decrypt(
        block,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None)) for block in blocks])


In [ ]:
def main(arguments):
    parser = argparse.ArgumentParser(
        formatter_class=argparse.RawDescriptionHelpFormatter)
    parser.add_argument(
        "-e", "--encrypt", help="Msg to be encrypted", type=str)
    parser.add_argument(
        "-d", "--decrypt", help="Msg to be decrypted", type=str)
    parser.add_argument(
        "-pub", "--public-key", help="File containing the public key", type=argparse.FileType('r'))
    parser.add_argument(
        "-prv", "--private-key", help="File containing the private key", type=argparse.FileType('r'))
    parser.add_argument(
        "-p", "--padding", help="File containing the private key", choices={"PKCS", "OAEP"})
    parser.add_argument(
        "-o", "--output", help="The output file", type=argparse.FileType('w'))

    args = parser.parse_args(arguments)

    if args.encrypt:
        if args.public_key:
            public_key = load_pem_public_key(
                args.public_key.read().encode())
        else:
            priv_key = generate_private_key(0x10001, 2048)
            with open("private_key.pem", "w") as priv_key_file:
                priv_key_file.write(priv_key.private_bytes(encoding=serialization.Encoding.PEM,
                                    format=serialization.PrivateFormat.PKCS8, encryption_algorithm=serialization.NoEncryption()).decode())

            public_key = priv_key.public_key()
            with open("public_key.pem", "w") as pub_key_file:
                pub_key_file.write(public_key.public_bytes(
                    encoding=serialization.Encoding.PEM, format=serialization.PublicFormat.SubjectPublicKeyInfo).decode())

        if args.padding == 'PKCS':
            encrypted = encrypt_PKCS(args.encrypt.encode(), public_key)
        else:
            encrypted = encrypt_OAEP(
                args.encrypt.encode(), public_key)

        if args.output:
            args.output.write(encrypted.hex())
        print(encrypted.hex())

    if args.decrypt:
        if args.private_key:
            private_key = load_pem_private_key(
                args.private_key.encode(), password=None)
        else:
            private_key = generate_private_key(0x10001, 2048)
            with open("private_key.pem", "w") as priv_key_file:
                priv_key_file.write(private_key.private_bytes(encoding=serialization.Encoding.PEM,
                                    format=serialization.PrivateFormat.PKCS8, encryption_algorithm=serialization.NoEncryption()).decode())

            public_key = private_key.public_key()
            with open("public_key.pem", "w") as pub_key_file:
                pub_key_file.write(public_key.public_bytes(
                    encoding=serialization.Encoding.PEM, format=serialization.PublicFormat.SubjectPublicKeyInfo).decode())

        if args.padding == 'PKCS':
            decrypted = decrypt_PKCS(
                bytes.fromhex(args.decrypt.read()),
                private_key)
        else:
            decrypted = decrypt_OAEP(
                bytes.fromhex(args.decrypt.read()),
                private_key)
        if args.output:
            args.output.write(decrypted.decode())
        print(decrypted.decode())




In [ ]:
sys.argv = [
    "",
    "-e", "prime",
    "-pub" , "InputFiles/RSA_cryptography_with_padding/public_key.pem",
    "-prv", "InputFiles/RSA_cryptography_with_padding/private_key.pem",
    "-p",  "PKCS",
    "-o", "OutputFiles/assign3_q5.txt"
    ]


main(sys.argv[1:])